# Some statistics for models

## ResNet

Number of trainable parameters for each ResNet depth, IE. the number of FLOPs (Floating Point Operations), which directly corresponds to complexity.

In [1]:
import torch
from tqdm import tqdm

In [2]:
from models.resnet import ResNet, ResidualBlock
from models.clinical_bert import bio_clinical_BERT
from models.linear_classifier import LinearClassifier

bert = bio_clinical_BERT()

resnet18 = ResNet(12, 5, 18, ResidualBlock)

c:\Users\ssjsi\Documents\Bachelors-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
resnet18_params = sum(p.numel() for p in resnet18.parameters() if p.requires_grad)
print(f"Number of trainable parameters in ResNet18: {resnet18_params:,}")

Number of trainable parameters in ResNet18: 4,111,488


In [4]:
from configs.ptb_xl_configs import Configs
from dataloader.ptb_xl import ptb_xl_data_generator
configs = Configs()
classifier = LinearClassifier(configs)
train_loader, val_loader, test_loader = ptb_xl_data_generator(configs, include_text=True)

Loading PTB-XL...
Finished loading PTB-XL in 0 minutes and 2 seconds.


In [5]:
from train_eval.trainer import Trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
trainer = Trainer()

In [6]:
resnet18 = resnet18.to(device)
classifier = classifier.to(device)

In [7]:
from train_eval.loss import Loss_CE
optimizer = torch.optim.Adam(resnet18.parameters(), lr=configs.learning_rate, weight_decay=configs.weight_decay)
criterion = Loss_CE()

losses = trainer.ecg_encoder_pre_train(resnet18, bert, train_loader, 3, optimizer, criterion, device)


Epoch 3 / 3: 100%|██████████| 102/102 [25:51<00:00, 15.21s/it]

Finished pre-training ResNet-18.


In [8]:
losses

[4.849668463071187, 4.849343157282062, 4.8497028631322525]